In [1]:
# Install required packages
# !pip install -r requirements.txt

In [2]:
# ! /get/notebook/lsy/env/bin/python -m pip install feather 

In [3]:
# Import necessary libraries
import pandas as pd
import os
import xgboost as xgb
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from pyarrow import feather
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import matplotlib.pyplot as plt
import numpy as np
import multiprocessing
import numpy as np
import feather

/get/smartmem_data/pyenv/lib/python3.11/site-packages/xgboost/core.py:265: FutureWarning: Your system has an old version of glibc (< 2.28). We will stop supporting Linux distros with glibc older than 2.28 after **May 31, 2025**. Please upgrade to a recent Linux distro (with glibc 2.28+) to use future versions of XGBoost.
Note: You have installed the 'manylinux2014' variant of XGBoost. Certain features such as GPU algorithms or federated learning are not available. To use these features, please upgrade to a recent Linux distro with glibc 2.28+, and install the 'manylinux_2_28' variant.
  warnings.warn(
/home/ais/.cache/matplotlib is not a writable directory
Matplotlib created a temporary cache directory at /tmp/matplotlib-xiekw1h9 because there was an issue with the default path (/home/ais/.cache/matplotlib); it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocess

In [4]:
import warnings
warnings.filterwarnings('ignore')

In [5]:
# Load ticket data
ticket = pd.read_csv('./ticket.csv')

In [6]:
# Display the ticket data
ticket.head()

,sn_name,alarm_time,sn_type
0,sn_4191,1704077850,A
1,sn_10692,1704140121,A
2,sn_7219,1704148156,A
3,sn_31281,1704151660,A
4,sn_8854,1704159849,A


In [7]:
import glob

In [8]:
# Load and process training data for Server Type A
def load_data():
    # trainset_A_path = './SmartHW-main/sample_data/server_type_A'
    trainset_A = []
    for file in glob.glob('/data1/smartmem/stage1_feather/*/*.feather'):
        if file.endswith('.feather'):
            df = pd.read_feather(file)  # Use pandas.read_feather()
            df['sn'] = os.path.basename(file).split('.')[0]  # Extract filename without extension
            trainset_A.append(df)
            

    # for filename in os.listdir(trainset_A_path):
    #     file_path = os.path.join(trainset_A_path, filename)
    #     if filename.endswith('.feather'):  # Check for Feather files
    #         df = pd.read_feather(file_path)  # Use pandas.read_feather()
    #         df['sn'] = os.path.splitext(filename)[0]  # Extract filename without extension
    #         trainset_A.append(df)

    trainset_A_df = pd.concat(trainset_A, ignore_index=True)
    return trainset_A_df

In [9]:
# trainset_A_df = load_data()

In [10]:
# trainset_A_df= trainset_A_df[trainset_A_df.LogTime<pd.to_datetime('2024-06-01').timestamp()]

In [11]:
# # Label the data based on anomalies
# trainset_A_df['anomaly'] = 0
# trainset_A_df.loc[trainset_A_df['sn'].isin(ticket['sn_name']), 'anomaly'] = 1

## Feature Engineering

### Error Counts Features

In [12]:
select_time_windows = [1,
                        5,
                       30,
                        60,
                       5*60,
                       10*60,
                       15*60,
                       30*60,
                       1*3600,
                       24*3600,
                      24*2*3600]

In [13]:
def preprocess_data(df):
    df = df.sort_values(['sn','LogTime'])
    df["time_index"] = np.ceil(df["LogTime"]/1)
    df['time_diff'] = df.groupby(['sn']).LogTime.diff()
    df['address'] = df['CpuId'].astype(str) + '-' + df['ChannelId'].astype(str) + '-' + df['RankId'].astype(str) + '-' + \
                    df['deviceID'].astype(str) + '-' + df['BankgroupId'].astype(str) + '-' + df['BankId'].astype(str) + '-' + \
                    df['ColumnId'].astype(str) + '-' + df['RowId'].astype(str)
    df['bank_column_key'] = df['CpuId'].astype(str) + '-' + df['ChannelId'].astype(str) + '-' + df['RankId'].astype(str) + '-' + \
                            df['deviceID'].astype(str) + '-' + df['BankgroupId'].astype(str) + '-' + df['BankId'].astype(str) + '-' + df['ColumnId'].astype(str)
    df['bank_row_key'] = df['CpuId'].astype(str) + '-' + df['ChannelId'].astype(str) + '-' + df['RankId'].astype(str) + '-' + \
                         df['deviceID'].astype(str) + '-' + df['BankgroupId'].astype(str) + '-' + df['BankId'].astype(str) + '-' + df['RowId'].astype(str)
    df['bank_key'] = df['CpuId'].astype(str) + '-' + df['ChannelId'].astype(str) + '-' + df['RankId'].astype(str) + '-' + \
                     df['deviceID'].astype(str) + '-' + df['BankgroupId'].astype(str) + '-' + df['BankId'].astype(str)
    df['Bankgroup_key'] = df['CpuId'].astype(str) + '-' + df['ChannelId'].astype(str) + '-' + df['RankId'].astype(str) + '-' + \
                     df['deviceID'].astype(str) + '-' + df['BankgroupId'].astype(str)
    df['device_key'] = df['CpuId'].astype(str) + '-' + df['ChannelId'].astype(str) + '-' + df['RankId'].astype(str) + '-' + \
                     df['deviceID'].astype(str) 
    df['rank_key'] = df['CpuId'].astype(str) + '-' + df['ChannelId'].astype(str) + '-' + df['RankId'].astype(str)
    df['CE_READ'] = df.error_type_full_name.str.contains('CE.READ')
    df['CE_SCRUB'] = df.error_type_full_name.str.contains('CE.SCRUB')
    df['CE_OTHER'] = (df.CE_READ==False)&(df.CE_SCRUB==False)
    
    df.RetryRdErrLogParity.fillna(0,inplace=True)
    df.RetryRdErrLog.fillna(0,inplace=True)
    df["parity_valid"] = df.RetryRdErrLog.astype(int)&0x0001 
    df['RetryRdErrLogParity'] = df[['RetryRdErrLogParity','parity_valid']].apply(lambda x:x[0] if x[1]==1 else 0,axis=1)

    return df

In [14]:
# trainset_A_df = preprocess_data(trainset_A_df)

In [15]:
def get_recent_data(df,row,time):
    start_time = row['LogTime'] - pd.Timedelta(seconds=time)
    end_time = row['LogTime']
    time_mask_ = (df['LogTime'] >= start_time) & (df['LogTime'] <= end_time)
    recent_data_ = df.loc[time_mask_]
    df.at[index,'error_count_CE.READ_{}'.format(time)] = recent_data_['error_type_full_name'].value_counts().get('CE.READ', 0)
    df.at[index, 'error_count_CE.SCRUB_{}'.format(time)] = recent_data_['error_type_full_name'].value_counts().get('CE.SCRUB', 0)
    

In [16]:
# sample = feather.read_dataframe('./type_A//sn_1.feather')

In [17]:
addr_cols = ['bank_column_key','bank_row_key','bank_key','Bankgroup_key','device_key','rank_key']

In [18]:
# sample['sn'] ='sn_1'

In [19]:
# sample =preprocess_data(sample)

In [20]:
# sample.groupby(['sn','time_index2','bank_row_key').LogTime.max().reset_index()

In [21]:
# '&'.join(['2','1-2-1-9.0-0-2-129330'])

In [22]:
# sample.groupby(['sn','time_index2','bank_row_key'])['time_diff'].agg([np.mean,np.std,np.ptp]).reset_index().rename(
#                                                                                             columns={'mean':'time_diff_mean{}'.format('&'.join(['2','1-2-1-9.0-0-2-129330'])),
#                                                                                                      'std':'time_diff_std{}'.format('&'.join(['2','1-2-1-9.0-0-2-129330'])),
#                                                                                                     'ptp':'time_diff_ptp{}'.format('&'.join(['2','1-2-1-9.0-0-2-129330'])),
#                                                                                                     })

In [23]:
# sample["time_diff"] = sample["LogTime"].max()-sample["LogTime"]

In [24]:
# sample["time_diff2"] = np.ceil(sample["time_diff"]/300)*300

In [25]:
# sample["time_diff2"] = pd.to_datetime(sample["time_diff2"], unit='s')
# sample["time_diff3"] = pd.to_datetime(sample["time_diff"], unit='s')

In [26]:
# sample["LogTime2"] = pd.to_datetime(sample["LogTime"], unit='s')

In [27]:
# sample["time_index"] = np.ceil(sample['LogTime'] / 300) * 300
# sample["time_index2"] = pd.to_datetime(sample["time_index"], unit='s')

In [28]:
# sample["time_index"] = np.ceil(sample['LogTime'] / 600) * 600
# sample["time_index3"] = pd.to_datetime(sample["time_index"], unit='s')

In [29]:
43440/300

144.8

In [30]:
# sample[['LogTime','LogTime2','time_diff','time_diff3','time_diff2','time_index2','time_index3']].head(30)

In [31]:
def calculate_counts_features_v2(tmp2,gdf,time):
    
    res= pd.DataFrame() 
    tmp1= gdf[['CE_READ','CE_SCRUB','CE_OTHER']].sum().reset_index().rename(columns={'CE_READ':'error_count_READ_{}'.format(time),
                                                                                                     'CE_SCRUB':'error_count_SCRUB_{}'.format(time),
                                                                                                                'CE_OTHER':'error_count_OTHER_{}'.format(time),
                                                                                                                    'error_type_full_name':'error_count_total_{}'.format(time)})
    tmp1['error_count_total_{}'.format(time)] = gdf.error_type_full_name.count().reset_index()['error_type_full_name']
    del tmp1['time_index']
    tmp1["LogTime"] = tmp2["LogTime"]
    return tmp1



### 以秒为最小时间单位，计算CE发生的时间间隔特征

In [32]:
def time_interval(x):
    x = x.tolist()
    return x[-1]-x[0]
def count_(x):
    return len(x)

In [33]:
# 1.发生CE的时间间隔
# 2.每个时间窗口内，平均间隔时间

def calculate_time_features_v2(tmp2,gdf,time):
    # res= pd.DataFrame() 
    # for time in select_time_windows[::-1]:
    #     df["time_index"] = np.ceil(df["LogTime"]/time)
    # df2 = df.dropna(subset=['time_diff'])
    tmp1 = gdf['time_diff'].agg([np.mean,np.std,np.ptp,time_interval]).reset_index().rename(
                                                                                            columns={'mean':'time_diff_mean{}'.format(time),
                                                                                                     'std':'time_diff_std{}'.format(time),
                                                                                                    'ptp':'time_diff_ptp{}'.format(time),
                                                                                                    'time_interval':'time_interval{}'.format(time)})
    # tmp2 = df2[['LogTime','time_index']].groupby('time_index').LogTime.max().reset_index()
    del tmp1['time_index']
    tmp1["LogTime"] = tmp2["LogTime"]
        # if res.empty:
        #     res = tmp1
        # else:
        #     res = res.merge(tmp1,on=['sn','LogTime'],how='right')
            
    return tmp1



### 计算一个时间窗口内的Fault Features

In [34]:
# Function to calculate fault features based on memory cell locations
def calculate_faults_features_v2(tmp3,gdf,time,addr_):
    res=pd.DataFrame()
    tmp1 = gdf[['CE_READ' ,'CE_SCRUB','CE_OTHER']].sum().reset_index().rename(columns={'CE_READ':'{}_fault_ce_n_{}'.format(addr_,time),
                                                                                        'CE_SCRUB':'{}_fault_sc_n_{}'.format(addr_,time),
                                                                                        'CE_OTHER':'{}_fault_ot_n_{}'.format(addr_,time)})
    tmp2 = gdf[['error_type_full_name']].nunique().reset_index().rename(columns={'error_type_full_name':'{}_total_fault_n_{}'.format(addr_,time)})
    del tmp1[addr_]
    # del tmp2[addr_]
    tmp1= tmp1.merge(tmp2,on=['sn','time_index'],how='left')
    del tmp1['time_index']
    tmp1["LogTime"] = tmp3["LogTime"]

     
        # if res.empty:
        #     res = tmp1
        # else:
        #     res = res.merge(tmp1,on=['sn','LogTime'],how='right')
    return tmp1
    




###LogTime计算CE计数的二级特征

In [35]:
def calculate_error_stastic_v2(tmp2,gdf,time,cols,addr):
    res= pd.DataFrame() 
    # for time in select_time_windows[::-1]:
    #     df["time_index"] = np.ceil(df["LogTime"]/time)
    # if time == 1:
    #     continue
    tmp1 = gdf[cols].agg([np.mean,np.std,np.ptp]).reset_index()
    tmp1.columns = ['_'.join(col) for col in tmp1.columns]

    for col in tmp1.columns:
        if col != "sn_" and col != "time_index_" and col != f"{addr}_":
            tmp1.rename(columns={col:f"{col}_{time}"},inplace=True)
        else:
            tmp1.rename(columns={col:col[0:len(col)-1]},inplace=True)

    # tmp2 = df[['LogTime','time_index']].groupby('time_index').LogTime.max().reset_index()
    del tmp1['time_index']

    tmp1["LogTime"] = tmp2["LogTime"]

    # if res.empty:
    #     res = tmp1
    # else:
    #     res = res.merge(tmp1,on=['sn','LogTime'],how='right')
    return tmp1

### 错误计数变化量

In [36]:
# count_features_diff1 = pd.concat([count_features[['sn','time_index']],count_features[count_features.columns.difference(['sn','time_index','LogTime'])].diff()],axis=1)

In [37]:
# faults_features_diff1 = pd.concat([faults_features[['sn','time_index']],faults_features[faults_features.columns.difference(['sn','time_index','LogTime'])].diff()],axis=1)

In [38]:
def get_level2_features_v2(tmp2,count_features,faults_features,time,addr_):
    cols1 = [f'error_count_READ_{time}', f'error_count_SCRUB_{time}', f'error_count_OTHER_{time}',f'error_count_total_{time}']
    err_count_statistic_features = calculate_error_stastic_v2(tmp2,count_features,time,cols1,addr_)
    cols2 = [f'{addr_}_fault_ce_n_{time}',f'{addr_}_fault_sc_n_{time}',f'{addr_}_fault_ot_n_{time}',f'{addr_}_total_fault_n_{time}',]
    faults_statistic_features = calculate_error_stastic_v2(tmp2,faults_features,time,cols2,addr_)
    return err_count_statistic_features,faults_statistic_features

In [39]:
def get_level2_diff_features_v2(count_features,faults_features):
    tmp = count_features[count_features.columns.difference(['sn','time_index','LogTime'])].diff()
    tmp.rename(columns=lambda x:f'{x}_d1',inplace=True)
    count_features_diff1 = pd.concat([count_features[['sn','LogTime']],tmp],axis=1)
    
    tmp=faults_features[faults_features.columns.difference(['sn','time_index','LogTime'])].diff()
    tmp.rename(columns=lambda x:f'{x}_d1',inplace=True)
    faults_features_diff1 = pd.concat([faults_features[['sn','LogTime']],tmp],axis=1)
    return count_features_diff1,faults_features_diff1

In [40]:
# _,_,a,b=get_level2_features(counts_features,faults_features)

In [41]:
# _.columns

### 计算时间窗口内发生CE风暴的数量

In [42]:
def _calculate_ce_storm_count(
    log_times: np.ndarray,
    ce_storm_interval_seconds: int = 60,
    ce_storm_count_threshold: int = 10,
) -> int:
    """
    计算 CE 风暴的数量

    CE 风暴定义:
    - 首先定义相邻 CE 日志: 若两个 CE 日志 LogTime 时间间隔 < 60s, 则为相邻日志;
    - 如果相邻日志的个数 >10, 则为发生 1 次 CE 风暴(注意: 如果相邻日志数量持续增长, 超过了 10, 则也只是记作 1 次 CE 风暴)

    :param log_times: 日志 LogTime 列表
    :param ce_storm_interval_seconds: CE 风暴的时间间隔阈值
    :param ce_storm_count_threshold: CE 风暴的数量阈值
    :return: CE风暴的数量
    """

    log_times = sorted(log_times)
    ce_storm_count = 0
    consecutive_count = 0

    for i in range(1, len(log_times)):
        if log_times[i] - log_times[i - 1] <= ce_storm_interval_seconds:
            consecutive_count += 1
        else:
            consecutive_count = 0
        if consecutive_count > ce_storm_count_threshold:
            ce_storm_count += 1
            consecutive_count = 0

    return ce_storm_count

    

In [43]:
def calculate_storm_features_v2(tmp2,gdf,time):
    res = pd.DataFrame()
    # for time in select_time_windows[::-1]:
    #     df["time_index"] = np.ceil(df["LogTime"]/time)
    tmp1 = gdf.LogTime.apply(lambda x:_calculate_ce_storm_count(x)).reset_index().rename(columns={'LogTime':'ce_storm_count_{}'.format(time)})

    # tmp2 = df[['LogTime','time_index']].groupby('time_index').LogTime.max().reset_index()
    tmp1["LogTime"] = tmp2["LogTime"]
    del tmp1['time_index']
        # if res.empty:
        #     res = tmp1
        # else:
        #     res = res.merge(tmp1,on=['sn','LogTime'],how='right')
    return tmp1



In [44]:
def compute_positive_areas(grid):
    rows, cols = len(grid), len(grid[0])
    visited = [[False] * cols for _ in range(rows)]
    areas = []  # 存储每个连通区域的面积

    def dfs(r, c):
        """ 深度优先搜索计算连通区域的面积 """
        if r < 0 or r >= rows or c < 0 or c >= cols:  # 越界
            return 0
        if visited[r][c] or grid[r][c] <= 0:  # 已访问或值不大于0
            return 0
        
        visited[r][c] = True  # 标记访问
        area = 1  # 当前点计入面积
        
        # 递归搜索四个方向
        area += dfs(r + 1, c)  # 下
        area += dfs(r - 1, c)  # 上
        area += dfs(r, c + 1)  # 右
        area += dfs(r, c - 1)  # 左
        
        return area

    for i in range(rows):
        for j in range(cols):
            if grid[i][j] > 0 and not visited[i][j]:  # 发现新的连通区域
                area = dfs(i, j)
                areas.append(area)  # 记录该区域的面积
    return areas  # 返回总面积



In [45]:
sorted([1,2,3])

[1, 2, 3]

In [46]:
def get_metrix_feature(data):
    burst = []
    dq = []
    
    for i in range(8):
        if sum(data[i]) >0:
            burst.append(i) 
            
    for j in range(4):
        if sum(data[:,j]) >0:
            dq.append(j)  
            
    max_burst = burst[-1] - burst[0] if len(burst) else 0
    max_dq = dq[-1] - dq[0] if len(dq) else 0
    
    err_dq_count = len(dq)
    err_burst_count = len(burst)

    err_row=[]
    for i in range(8):
        if sum(data[i]>0) >1:
            err_row.append(sum(data[i]>0))
    err_raw_count = len(err_row)
    
  
    err_raw_count_max = sorted(err_row)[-1] if len(err_row)>0 else 0

    err_col=[]
    for j in range(4):
        if sum(data[:,j]>0) >1:
            err_col.append(sum(data[:,j]>0)) 
    err_col_count = len(err_col)

    err_col_count_max = sorted(err_col)[-1] if len(err_col)>0 else 0

    areas = compute_positive_areas(data.tolist())

    max_adj_areas = max(areas) if len(areas) else 0 
    total_areas = sum(areas) if len(areas) else 0
    areas_count = len(areas)
    return [max_adj_areas,total_areas,areas_count,max_burst,max_dq,err_raw_count,err_col_count,err_dq_count,err_burst_count,err_raw_count_max,err_col_count_max]

In [47]:
def process_metrix(paritys):
    res = np.zeros([8,4])
    def _process(parity):
        bin_parity = bin(int(parity))[2:].zfill(32)
        return np.array([[int(j) for j in bin_parity[i : i + 4]] for i in range(0, 32, 4)])
    for parity in paritys:
        res += _process(parity)
    return get_metrix_feature(res)

In [48]:
def calculate_bit_features2_v2(tmp2,gdf,time):
    res = pd.DataFrame()
    # print(gdf.RetryRdErrLogParity.apply(lambda x:pd.DataFrame([process_metrix(x)])).reset_index())
    tmp1= gdf.RetryRdErrLogParity.apply(lambda x:pd.DataFrame([process_metrix(x)])).reset_index().drop(columns=['level_3']).rename(
                                                    columns=dict(zip([0,1,2,3,4,5,6,7,8,9,10],
                                                           ['max_adj_areas_{}'.format(time),
                                                            'total_areas_{}'.format(time),
                                                            'areas_count_{}'.format(time),
                                                           'max_burst_{}'.format(time),
                                                            'max_dq_{}'.format(time),
                                                            'err_raw_count_{}'.format(time),
                                                            'err_col_count_{}'.format(time),
                                                            'err_dq_count_{}'.format(time),
                                                            'err_burst_count_{}'.format(time),
                                                            'err_raw_count_max_{}'.format(time),
                                                            'err_col_count_max_{}'.format(time),])))
    tmp1["LogTime"] = tmp2["LogTime"]
    return tmp1



In [49]:
def calculate_addr_features_v2(tmp2,gdf,time):
    res= pd.DataFrame() 
    tmp1 = gdf[['MciAddr','RetryRdErrLogParity']].nunique().reset_index().rename(columns={'MciAddr':'MciAddr_nunique{}'.format(time),
                                                                                                     'RetryRdErrLogParity':'Parity_nuique{}'.format(time),                                                                                    })
    del tmp1['time_index']

    tmp1["LogTime"] = tmp2["LogTime"]

    return tmp1

### Combine Features for Modeling

In [50]:
def _merge_df(*args):
    features = args[0]
    for df in args[1:]:
        try:
            features = features.merge(df,on=['sn','LogTime'],how='left')
        except:
            import traceback
            print(traceback.print_exc())
            print(df.columns)
    return features

In [51]:
# faults_features_diff1.columns.tolist()

In [52]:
import threading
import multiprocessing
from multiprocessing import Pool


In [54]:
def process_single_sn(sn_file,):
    trainset_A_df = pd.read_feather(os.path.join(data_path,sn_file))  
    trainset_A_df['sn'] = os.path.basename(sn_file).split('.')[0]
    
    trainset_A_df = preprocess_data(trainset_A_df)
    
    train = trainset_A_df[trainset_A_df.LogTime<pd.to_datetime('2024-06-01').timestamp()]
    test = trainset_A_df[trainset_A_df.LogTime>pd.to_datetime('2024-06-01').timestamp()]
 
    
    train = train[train.LogTime>(train.LogTime.max() - max(select_time_windows))]
    test = test[test.LogTime>(test.LogTime.max() - max(select_time_windows))]
    
    if not os.path.exists(os.path.join(feature_path,'train')):
        os.makedirs(os.path.join(feature_path,'train'))
    if not os.path.exists(os.path.join(feature_path,'test')):
        os.makedirs(os.path.join(feature_path,'test'))
        
    if not os.path.exists(os.path.join(os.path.join(feature_path,'train'),sn_file.split('.')[0])):
        os.makedirs(os.path.join(os.path.join(feature_path,'train'),sn_file.split('.')[0]))
    
    if not os.path.exists(os.path.join(os.path.join(feature_path,'test'),sn_file.split('.')[0])):
        os.makedirs(os.path.join(os.path.join(feature_path,'test'),sn_file.split('.')[0]))
    addr_col_ = ['bank_column_key','bank_row_key','bank_key','Bankgroup_key','device_key','rank_key'][::-1]
    
    def _cal_(sn_file,trainset_A_df,feature_path):
        # error_bit_feature = trainset_A_df.RetryRdErrLogParity.apply((lambda x:_get_bit_dq_burst_info(x))).rename(columns=dict(zip([0,1,2,3,4],['bit_count', 'dq_count', 'burst_count', 'max_dq_interval', 'max_burst_interval'])))
        # error_bit_feature = trainset_A_df[['sn','LogTime']].merge(error_bit_feature,left_index=True,right_index=True)
        
        for time_ in select_time_windows[::-1]:
            for addr_index in  range(len(['bank_column_key','bank_row_key','bank_key','Bankgroup_key','device_key','rank_key'])):
                addr_ = addr_col_[addr_index]
                
                
                trainset_A_df["time_index"] = np.ceil(trainset_A_df["LogTime"]/time_)*time_
                gdfcol = ['sn','time_index']+[addr_]
                
                tmp2 = trainset_A_df[['LogTime']+gdfcol].groupby('time_index').LogTime.max().reset_index()

                cols = [addr_,str(time_)]
    
                gdf = trainset_A_df.groupby(gdfcol)
                counts_features =calculate_counts_features_v2(tmp2,gdf,'&'.join(cols))
                
                
                # del counts_features[addr_]
    
                error_bit_feature2 = calculate_bit_features2_v2(tmp2,gdf,'&'.join(cols))
           
                del error_bit_feature2[addr_]
                


            
                faults_features =calculate_faults_features_v2(tmp2,gdf,'&'.join(cols),addr_)
                
                
                addr_feature = calculate_addr_features_v2(tmp2,gdf,'&'.join(cols))
                
                del addr_feature[addr_]
                    
                ce_storm_feature =calculate_storm_features_v2(tmp2,gdf,'&'.join(cols))
                
                del ce_storm_feature[addr_]

                
                    
                
                trainset_A_df['time_diff'] = trainset_A_df.time_diff.fillna(0) 
         
                gdf = trainset_A_df.groupby(gdfcol)
                time_features = calculate_time_features_v2(tmp2,gdf,'&'.join(cols))
                
                del time_features[addr_]
                    
                
                
                counts_features["time_index"] = np.ceil(counts_features["LogTime"]/time_)*time_
                faults_features["time_index"] = np.ceil(faults_features["LogTime"]/time_)*time_
                
                counts_features_gdf = counts_features.groupby(gdfcol)
                faults_features_gdf = faults_features.groupby(gdfcol)

                del faults_features[addr_]
                del counts_features[addr_]
        
                err_count_statistic_features,faults_statistic_features= get_level2_features_v2(tmp2,counts_features_gdf,faults_features_gdf,'&'.join(cols),addr_)
                del err_count_statistic_features[addr_]
                del faults_statistic_features[addr_]
                    
                count_features_diff1,faults_features_diff1 = get_level2_diff_features_v2(counts_features,faults_features)
                del counts_features["time_index"]
                del faults_features["time_index"]
                
                def __func():
                    features = _merge_df(time_features,
                                     faults_features,
                                         error_bit_feature2,
                                     addr_feature,
                                    counts_features,
                                    ce_storm_feature,
                                         err_count_statistic_features,
                                         faults_statistic_features,
                                         count_features_diff1,
                                         faults_features_diff1)
                    fileanme = '&'.join(cols)
                
                    feather.write_dataframe(
                            features,
                            os.path.join(feature_path, sn_file.split('.')[0],f'{sn_file.replace("csv", "feather")}_{fileanme}'),
                        )
                import multiprocessing
                p = multiprocessing.Process(target=__func,args=())
                p.start()
                # __func()
            
               
        
    if train.shape[0]: _cal_(sn_file,train,os.path.join(feature_path,'train'))
    if test.shape[0]:_cal_(sn_file,test,os.path.join(feature_path,'test'))
    # print(sn_file,trainset_A_df.shape,time.time()-start,time.time())
def worker_initializer():
    # 设置子进程为非守护进程
    multiprocessing.current_process().daemon = False
    
def process_all_sn(data_path,feature_path) :
        """
        处理所有 sn 文件, 并保存特征, 支持多进程处理以提高效率
        """

        sn_files = os.listdir(data_path)
        sn_files.sort()
        with Pool(multiprocessing.cpu_count(), initializer=worker_initializer) as pool:
            list(
                tqdm(
                    pool.imap(process_single_sn,sn_files),
                    total=len(sn_files),
                    desc="Generating features",
                )
            )
       

In [ ]:
# %%time
# import time
# start = time.time()
# data_path="type_A/"
# feature_path="./feature"
# process_single_sn("sn_22027.feather")


In [ ]:
# feather.read_feather('./feature/type_B/sn_9.feather').columns.tolist()


In [ ]:
from multiprocessing import Pool

In [ ]:
0x0001

In [ ]:
# df

In [ ]:
# dddd

In [ ]:
%%time
# data_path="./SmartHW-main/sample_data/server_type_A/"
data_path="type_A/"
feature_path="./feature/type_A/"
process_all_sn(data_path,feature_path)

Generating features:   0%|          | 0/56403 [00:00<?, ?it/s]

In [ ]:
# glob.glob(f'./feature/*/train/*')[:120]

In [ ]:
# ddd

In [ ]:
%%time
data_path="type_B/"
feature_path="./feature/type_B/"
process_all_sn(data_path,feature_path)

In [ ]:
# os.listdir('./feature/type_A/train/sn_22027/')

In [ ]:
only_test = []
for sn_file in glob.glob(f'./feature/*/train/*'):
    if len(glob.glob(f'{sn_file}/*'))==0:
        only_test.append(sn_file)

In [ ]:
len(only_test)

In [ ]:
testlist=[]
for sn_file in glob.glob(f'./feature/*/test/*'):
    if len(glob.glob(f'{sn_file}/*'))!=0:
        testlist.append(sn_file)

In [ ]:
len(testlist)

In [ ]:
trainlist=[]
for sn_file in glob.glob(f'./feature/*/train/*'):
    if len(glob.glob(f'{sn_file}/*'))!=0:
        trainlist.append(sn_file)

In [ ]:
len(trainlist)

In [ ]:
def read_signal_data(sn_file):
    df = pd.DataFrame()
    trainset_A = []
    if os.path.exists( os.path.join(sn_file,f'{sn_file.split(os.sep)[-1]}_total')):
        df = pd.read_feather(os.path.join(sn_file,f'{sn_file.split(os.sep)[-1]}_total'))  
        df['sn'] = sn_file.split(os.sep)[-1]
        return df
    else:
        for file in glob.glob(f'{sn_file}/*'):
            try:
                df = pd.read_feather(file)  # Use pandas.read_feather()
                df['sn'] = os.path.basename(file).split('.')[0]  # Extract filename without extension
                trainset_A.append(df)
                df = _merge_df(*trainset_A)
            except:
                import traceback
                print(file,traceback.print_exc())
        if df.shape[0]:
                feather.write_dataframe(
                    df,
                    os.path.join(sn_file,f'{sn_file.split(os.sep)[-1]}_total'),
                )
    return df

def read_all_data(path='./feature/*',mode='train'):
    print(f'{path}/{mode}/*')
    results = []
    resdf = []
    with multiprocessing.Pool(multiprocessing.cpu_count()) as pool:
        for sn_file in glob.glob(f'{path}/{mode}/*'):
            p = pool.apply_async(read_signal_data, (sn_file,))
            results.append(p)
        for p in results:
            tmp = p.get()
            if tmp.shape[0]:resdf.append(tmp)
    results = pd.concat(resdf)
    return results


In [ ]:
# print(len(glob.glob(f'./feature/*/train/*')[:500]))

In [ ]:
%%time
traindata = read_all_data(path='./feature/*',mode='train')

In [ ]:
# bit_count, dq_count, burst_count, max_dq_interval, max_burst_interva/l

In [ ]:
bit_count_cols = traindata.columns[traindata.columns.str.contains('bit_count')].tolist()
dq_count_cols = traindata.columns[traindata.columns.str.contains('dq_count')].tolist()
burst_count_cols = traindata.columns[traindata.columns.str.contains('burst_count')].tolist()
max_dq_interval_cols = traindata.columns[traindata.columns.str.contains('max_dq_interval')].tolist()
max_burst_interva_cols = traindata.columns[traindata.columns.str.contains('max_burst_interva')].tolist()

In [ ]:
# select_cols = traindata.columns.difference(bit_count_cols+dq_count_cols+burst_count_cols+max_dq_interval_cols+max_burst_interva_cols+['LogTime', 'anomaly', 'sn'])

In [ ]:
# ssssss

In [ ]:
# traindata.sn.unique()

In [ ]:
# %%time
# testdata = read_all_data(path='./feature/*',mode='test')

In [ ]:
# lll

In [ ]:
# Label the data based on anomalies
traindata['anomaly'] = 0
traindata.loc[traindata['sn'].isin(ticket['sn_name']), 'anomaly'] = 1

In [ ]:
# %%time
# X = traindata[traindata.columns.difference(['sn','LogTime','time_index'])]
# y = traindata[['sn','LogTime','anomaly']]

In [ ]:
# traindata.sn.unique()[:50]

In [ ]:
# y.anomaly.value_counts()

In [ ]:
# trainset_A_df.shape,features.shape

In [ ]:
# train_test_split(X, y, test_size=0.1, random_state=42,stratify=y['anomaly'])

In [ ]:
# %%time
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42,stratify=y['anomaly'])

In [ ]:
# traindata[traindata.sn.isin(traindata[traindata.sn.isin(ticket['sn_name'])].sn.drop_duplicates().sample(frac=0.1))].shape

In [ ]:
def get_train_test(traindata,i=2025):
  
    bad_sn = traindata[traindata.sn.isin(ticket['sn_name'])].sn.drop_duplicates()
    good_sn = traindata[~traindata.sn.isin(ticket['sn_name'])].sn.drop_duplicates()
    
    test_good_sn=good_sn.sample(frac=0.2,random_state=i)
    test_bad_sn = bad_sn.sample(frac=0.2,random_state=i)

    test_sn = pd.concat([test_good_sn,test_bad_sn])
    
    test_data = traindata[traindata.sn.isin(test_sn)]
    train_data = traindata[~traindata.sn.isin(test_sn)]
    
    X_train = train_data[train_data.columns.difference(['sn','LogTime','time_index','anomaly'])]
    y_train = train_data[['sn','LogTime','anomaly']]
    X_test = test_data[test_data.columns.difference(['sn','LogTime','time_index','anomaly'])]
    y_test = test_data[['sn','LogTime','anomaly']]
    
    return X_train, X_test, y_train, y_test


In [ ]:
%%time
X_train, X_test, y_train, y_test = get_train_test(traindata)

In [ ]:
# sum(y_test.sn.isin(y_train.sn))

In [ ]:
# test_data = traindata[traindata.isin(test_sn)]
# train_data = traindata[~traindata.isin(test_sn)]

In [ ]:
# X_train = train_data[train_data.columns.difference(['sn','LogTime','time_index'])]
# y_train = train_data[['sn','LogTime','anomaly']]
# X_test = test_data[test_data.columns.difference(['sn','LogTime','time_index'])]
# y_test = test_data[['sn','LogTime','anomaly']]

In [ ]:
from scipy.misc import derivative
from scipy import stats

In [ ]:
traindata.anomaly.value_counts()

In [ ]:
# 136342/1904280

In [ ]:
%%time
def focal_loss_lgb(y_pred, dtrain, alpha=0.9, gamma=2):
    a,g = alpha, gamma
    y_true = dtrain.label
    def fl(x,t):
        p = 1/(1+np.exp(-x))
        return -( a*t + (1-a)*(1-t) ) * (( 1 - ( t*p + (1-t)*(1-p)) )**g) * ( t*np.log(p)+(1-t)*np.log(1-p) )
    partial_fl = lambda x: fl(x, y_true)
    grad = derivative(partial_fl, y_pred, n=1, dx=1e-6)
    hess = derivative(partial_fl, y_pred, n=2, dx=1e-6)
    return grad, hess

def focal_loss_lgb_eval(y_pred, dtrain, alpha=0.97, gamma=2):
    a,g = alpha, gamma
    y_true = dtrain.label
    p = 1/(1+np.exp(-y_pred))
    loss = -( a*y_true + (1-a)*(1-y_true) ) * (( 1 - ( y_true*p + (1-y_true)*(1-p)) )**g) * ( y_true*np.log(p)+(1-y_true)*np.log(1-p) )
    return 'focal_loss', np.mean(loss), False

In [ ]:
import lightgbm

In [ ]:
params = {'learning_rate':0.1,
              'num_leaves':127,
              'subsample':1,
              'colsample_bytree':1,
              'random_state':4,            
              'metric':'focal_loss_lgb_eval_error',
          'objective':focal_loss_lgb,
              'num_threads':-1,       
          'verbose':1,
          'early_stop':True,
        }
lgb_train=lightgbm.Dataset(X_train[select_cols], label=y_train['anomaly'])
# lgb_test = xgb.DMatrix(X_test, label=y_test['anomaly'], enable_categorical=True)
lgbm = lightgbm.train(
    params,
    lgb_train,
    valid_sets = lgb_train,
    feval=focal_loss_lgb_eval_error,
    num_boost_round = 50,
    # early_stopping_rounds=50,
    )

In [ ]:
# lgb_test = lightgbm.Dataset(X_test, label=y_test['anomaly'])

In [ ]:
# lgbm.predict(X_test)

In [ ]:
# dtrain_xgb.get_label()

### Model Training with XGBoost

In [ ]:

# %%time
# dtrain_xgb = xgb.DMatrix(X_train, label=y_train['anomaly'], enable_categorical=True)
# dtest_xgb = xgb.DMatrix(X_test, label=y_test['anomaly'], enable_categorical=True)

# # XGBoost model parameters
# params_xgb = {
#     'objective': 'binary:logistic',
#     'tree_method': 'hist',
#     'eval_metric': 'logloss',
#     'random_state': 42,
#     # 'verbose':0,
#     # 'num_leaves':127,
#     # 'subsample':1,
#     # 'colsample_bytree':1,
#     # 'random_state':4, 
#     # 'learning_rate':0.06,
# }

# # Custom callback for progress bar
# class XGBoostProgressCallback(xgb.callback.TrainingCallback):
#     def __init__(self, total):
#         self.pbar = tqdm(total=total, desc="XGBoost Training Progress")

#     def after_iteration(self, model, epoch, evals_log):
#         self.pbar.update(1)
#         if epoch + 1 == self.pbar.total:
#             self.pbar.close()
#         return False
# def custom_metric(predt, dtrain):
#     label = dtrain.get_label()
#     error = np.mean((predt - label) ** 2) 
#     return "custom_error", error

# early_stop = xgb.callback.EarlyStopping(rounds=10, 
#                                         metric_name="custom_error", 
#                                         data_name="test")

# # Train the model with progress bar
# num_boost_round = 100
# evals_result_xgb = {}
# bst_xgb = xgb.train(params_xgb, dtrain_xgb, 
#                     num_boost_round=num_boost_round,
#                     evals=[(dtest_xgb, 'test')],
#                     evals_result=evals_result_xgb,
#                     early_stopping_rounds=True,
#                     # obj=focal_loss_lgb
#                     )

In [ ]:
# Model evaluation
# y_pred_xgb = bst_xgb.predict(dtest_xgb)
y_pred_xgb =lgbm.predict(X_test[select_cols])
# y_pred_xgb_binary = [1 if y > 0.5 else 0 for y in y_pred_xgb]


In [ ]:
get_best_score(y_test,ticket,lgbm,select_cols)

In [ ]:
def get_best_score(y_test,ticket,lgbm,select_cols):
    y_pred_xgb =lgbm.predict(X_test[select_cols])
    subtest = y_test
    subtest = subtest.merge(ticket,left_on=['sn'],right_on=['sn_name'],how='left')
    subtest['y_pred_xgb'] = y_pred_xgb
    res = []
    for i in range(1,100,1):
        sub = subtest
        sub['pred'] = sub['y_pred_xgb'] > (i/100)
        sub['pred_t'] = (sub[sub.pred==1].alarm_time - sub[sub.pred==1].LogTime>15*60) &(sub[sub.pred==1].alarm_time - sub[sub.pred==1].LogTime<7*24*60*60+15*60)
        try:
            precise = sub[sub.pred_t==True].sn.nunique()/sub[sub.pred==1].sn.nunique()
        except:
            precise=0
        try:
            recall = sub[sub.pred_t==True].sn.nunique()/sub[sub.anomaly==1].sn.nunique()
        except:
            recall = 0
        try:
            f1 = 2*precise*recall/(precise+recall)
        except:
            f1=0
        res.append({"precise":precise,"recall":recall,"f1":f1,"threshold":i/100,'ntpp':tp,'npp':sub[sub.pred==1].sn.nunique(),"ntpr":sub[sub.anomaly==1].sn.nunique()})
    tmp = pd.DataFrame(res).sort_values('f1').tail(1)
    # .tail(1)
    # try:
    #     f.write('平均分为:{},probability:{}\n'.format(tmp.f1.values(0),tmp.threshold.values(0)))
    #     f.flush()
    # except Exception:
    #     print('平均分为:{},probability:{}\n'.format(tmp.f1.values(0),tmp.threshold.values(0)))

    
    try:
        f.write(str(tmp.to_dict("index"))+'\n')
        f.flush()
    except Exception:
        print(tmp.to_dict("index"))
    return tmp.f1.values[0]
     

In [ ]:
pd.DataFrame(res).sort_values('f1').tail(1).f1.values[0]

In [ ]:
# pd.DataFrame(res).sort_values('f1')

In [ ]:
# pd.DataFrame(res).sort_values('f1')

In [ ]:
# ffffffff

In [ ]:
from lightgbm import LGBMClassifier

In [ ]:

a,g = 0.9, 2
seed = 7
test_size = 0.1

def focal_loss_lgb(y_true, y_pred):
    def fl(x,t):
        p = 1/(1+np.exp(-x))
        return -( a*t + (1-a)*(1-t) ) * (( 1 - ( t*p + (1-t)*(1-p)) )**g) * ( t*np.log(p)+(1-t)*np.log(1-p) )
    partial_fl = lambda x: fl(x, y_true)
    grad = derivative(partial_fl, y_pred, n=1, dx=1e-6)
    hess = derivative(partial_fl, y_pred, n=2, dx=1e-6)
    return grad, hess
  
def focal_loss_lgb_eval(y_true, y_pred):
    p = 1/(1+np.exp(-y_pred))
    loss = -( a*y_true + (1-a)*(1-y_true) ) * (( 1 - ( y_true*p + (1-y_true)*(1-p)) )**g) * ( y_true*np.log(p)+(1-y_true)*np.log(1-p) )
    return 'focal_loss', np.mean(loss), False
# split data into train and test sets
def ga_train_clf(features_name, X_train, X_test, y_train, y_test):
    clf = LGBMClassifier(
        learning_rate=0.05,
        n_estimators=50,
        num_leaves=127,
        subsample=0.7,
        colsample_bytree=0.8,
        random_state=2025,
        metric=['auc'],
        n_jobs = -1,
        objective=focal_loss_lgb,
        # categorical_column ='model'
        # earl

    )
    # X_train, X_test, y_train, y_test = train_test_split(train_x[features_name], train_y, test_size=0.2, random_state=7)



    clf.fit(
        X_train[features_name], y_train['anomaly'],
        # eval_set=[(X_train[features_name], y_train),(X_test[features_name], y_test)],
        # verbose=0,  
        eval_metric='auc',
    )
    return clf

In [ ]:
 ga_train_clf(select_cols, X_train, X_test, y_train, y_test)

In [ ]:
f = open('./test_lgb_v2.txt','w')
used_columns = X_train.columns.difference(['sn','LogTime','time_index','anomaly'])

In [ ]:
SCORE_NONE = -1

class Life(object):
      """个体类"""
      def __init__(self, aGene=None):
            self.gene = aGene
            self.score = SCORE_NONE  # 初始化生命值  #

In [ ]:
import copy
import random
#from Life import Life
import numpy as np

class GA(object):
    """遗传算法类"""

    def __init__(self, aCrossRate, aMutationRage, aLifeCount, aGeneLenght, aMatchFun=lambda life: 1):
        self.croessRate = aCrossRate  # 交叉概率 #
        self.mutationRate = aMutationRage  # 突变概率 #
        self.lifeCount = aLifeCount   # 个体数 #
        self.geneLenght = aGeneLenght  # 基因长度 #
        self.matchFun = aMatchFun  # 适配函数
        self.lives = []  # 种群
        # self.best = None  # 保存这一代中最好的个体
        self.best = Life(np.random.randint(0, 2, self.geneLenght))  # 保存这一代中最好的个体

        self.gene = np.random.randint(0, 2, self.geneLenght)  # 保存全局最好的个体 #
        self.score = -1   # 保存全局最高的适应度 #

        self.generation = 0  # 第几代 #
        self.crossCount = 0  # 交叉数量 #
        self.mutationCount = 0  # 突变个数 #
        self.bounds = 0.0  # 适配值之和，用于选择时计算概率
        self.initPopulation()  # 初始化种群 #

    def initPopulation(self):
        """初始化种群"""
        self.lives = []
        count = 0
        while count < self.lifeCount:
            gene = np.random.randint(0, 2, self.geneLenght)
            life = Life(gene)
            random.shuffle(gene)  # 随机洗牌 #
            self.lives.append(life)
            count += 1

    def judge(self):
        """评估，计算每一个个体的适配值"""
        self.bounds = 0.0
        # self.best = self.lives[0]
        self.best.score = copy.deepcopy(self.score)  ####
        self.best.gene = copy.deepcopy(self.gene)  ####
        for life in self.lives:
            life.score = self.matchFun(life)
            self.bounds += life.score
            if self.best.score < life.score:     # score为auc 越大越好 #
                self.best = life

        if self.score < self.best.score:                          ####
            self.score = copy.deepcopy(self.best.score)           ####
            self.gene = copy.deepcopy(self.best.gene)             ####

        # self.best.score = copy.deepcopy(self.score)               ####
        # self.best.gene = copy.deepcopy(self.gene)                 ####

    def cross(self, parent1, parent2):
        """
        函数功能：交叉
        """
        index1 = random.randint(0, self.geneLenght - 1)  # 随机生成突变起始位置 #
        index2 = random.randint(index1, self.geneLenght - 1)  # 随机生成突变终止位置 #

        for index in range(len(parent1.gene)):
            if (index >= index1) and (index <= index2):
                parent1.gene[index], parent2.gene[index] = parent2.gene[index], parent1.gene[index]

        self.crossCount += 1
        return parent1.gene

    def mutation(self, gene):
        """突变"""
        index1 = random.randint(0, self.geneLenght - 1)
        index2 = random.randint(0, self.geneLenght - 1)
        # 随机选择两个位置的基因交换--变异 #
        newGene = gene[:]  # 产生一个新的基因序列，以免变异的时候影响父种群
        newGene[index1], newGene[index2] = newGene[index2], newGene[index1]
        self.mutationCount += 1
        return newGene

    def getOne(self):
        """选择一个个体"""
        r = random.uniform(0, self.bounds)
        for life in self.lives:
            r -= life.score
            if r <= 0:
                return life

        raise Exception("选择错误", self.bounds)

    def newChild(self):
        """产生新的后代"""
        parent1 = self.getOne()
        rate = random.random()

        # 按概率交叉 #
        if rate < self.croessRate:
            # 交叉 #
            parent2 = self.getOne()
            gene = self.cross(parent1, parent2)
        else:
            gene = parent1.gene

        # 按概率突变 #
        rate = random.random()
        if rate < self.mutationRate:
            gene = self.mutation(gene)

        return Life(gene)

    def next(self):
        """产生下一代"""
        self.judge()
        newLives = []
        newLives.append(self.best)  # 把最好的个体加入下一代 #
        newLives[0].gene = copy.deepcopy(self.gene)
        newLives[0].score = copy.deepcopy(self.score)
        while len(newLives) < self.lifeCount:
            newLives.append(self.newChild())
        self.lives = newLives
        self.generation += 1

In [ ]:
import random
import math
import numpy as np
import lightgbm as lgb
import pandas as pd
#from Genetic_algorithm import GA
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score


class FeatureSelection(object):
    def __init__(self, aLifeCount=10):
        self.columns = used_columns
        self.lifeCount = aLifeCount
        self.ga = GA(aCrossRate=0.7,
                     aMutationRage=0.8,
                     aLifeCount=self.lifeCount,
                     aGeneLenght=len(self.columns),
                     aMatchFun=self.matchFun())

    def auc_score(self, order):
        #print(order)
        features = self.columns
        features_name = []
        for index in range(len(order)):
            if order[index] == 1:
                features_name.append(features[index])
#         print(features_name)
        f.write(str(features_name)+'\n')
        f.flush()
    
        # clf = ga_train_clf(features_name, train_x, train_y)
        clf = ga_train_clf(features_name, X_train, X_test, y_train, y_test)
        best_score = get_best_score(y_test,ticket,clf,features_name)
        
        f.write('平均分:{}\n'.format(str(best_score)))      
        f.flush()
        return best_score

    def matchFun(self):
        return lambda life: self.auc_score(life.gene)

    def run(self, n=0):
        distance_list = []
        generate = [index for index in range(1, n + 1)]
        while n > 0:
            self.ga.next()
            distance = self.auc_score(self.ga.best.gene)
            #distance = self.ga.score                      ####
            distance_list.append(distance)
            print(("第%d代 : 当前最好特征组合的线下验证结果为：%f") % (self.ga.generation, distance))
            f.write(("第%d代 : 当前最好特征组合的线下验证结果为：%f\n" ) % (self.ga.generation, distance))
            f.flush()
            n -= 1
# 
        print('当前最好特征组合:')
        f.write('当前最好特征组合:\n')
        f.flush()
        string = []
        flag = 0
        features = self.columns[1:]
        for index in self.ga.gene:                                  ####
            if index == 1:
                string.append(features[flag])
            flag += 1
#         print(string)
        f.write(str(string)+'\n')
        f.write('线下最高为auc：{}\n'.format(self.ga.score))
        print('线下最高为auc：{}'.format(self.ga.score))   
        f.flush()
        ####

        '''画图函数'''
        plt.plot(generate, distance_list)
        plt.xlabel('generation')
        plt.ylabel('distance')
        plt.title('generation--auc-score')
        plt.show()


def main():
    fs = FeatureSelection(aLifeCount=20)
    rounds = 100    # 算法迭代次数 #
    fs.run(rounds)


if __name__ == '__main__':
    main()
    f.close()

In [ ]:
import collections
from collections import defaultdict
sn_type_list = []
for i in glob.glob('/data1/smartmem/stage1_feather/*/*.feather'):
    type_ = i.split(os.sep)[-2].split('_')[1]
    sn_ = i.split(os.sep)[-1].split('.')[0]
    sn_type_list.append({'sn':sn_,'serial_number_type':type_})

In [ ]:
pd.read_csv('submission.csv').columns

In [ ]:
sn_type = pd.DataFrame(sn_type_list)

In [ ]:
sn_type

In [ ]:

# dtest = xgb.DMatrix(testdata[X_test.columns], enable_categorical=True)
# test_data_y =  bst_xgb.predict(dtest)
# test_data_y = [1 if y > 0.3 else 0 for y in test_data_y]
# submit = testdata[['sn','LogTime']].merge(sn_type,on='sn')
# testdata.columns = ['sn_name', 'prediction_timestamp', 'serial_number_type']
# testdata.to_csv('./submission.csv',index=False)

In [ ]:
# 53/167

In [ ]:
# import datetime


In [ ]:
# ticket['time'] =ticket.alarm_time - 15*60 -7*24*3600

In [ ]:
# sub = sub.merge(ticket,left_on=['sn'],right_on=['sn_name'],how='left')

In [ ]:
# sub.shape

In [ ]:
# tp = sum((sub[sub.pred==1].alarm_time - sub[sub.pred==1].LogTime>15*60) &(sub[sub.pred==1].alarm_time - sub[sub.pred==1].LogTime<7*24*60*60+15*60))

In [ ]:
# sub[sub.pred==1].shape[0],sub[sub.anomaly==1].shape[0]

In [ ]:
# precise = tp/sub[sub.pred==1].shape[0]

In [ ]:
# recall = tp/sub[sub.anomaly==1].shape[0]

In [ ]:
# f1 = 2*precise*recall/(precise+recall)

In [ ]:
# f1

In [ ]:
precise

In [ ]:
recall

In [ ]:
# Plot feature importances for XGBoost
xgb.plot_importance(bst_xgb)
plt.show()

In [ ]:
cols = []

In [ ]:
for i,j in zip(lgbm.feature_name(),lgbm.feature_importance()):
    if j>0:
        print(i,j)
        cols.append(i)